In [ ]:
from datasets import load_dataset

dataset_inpaintings = load_dataset("imagefolder", data_dir="./inpaintings_dataset")
dataset_style_transfer = load_dataset("imagefolder", data_dir="./style_transfer_dataset")

In [2]:
dataset_inpaintings

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 7120
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 3006
    })
})

In [3]:
dataset_style_transfer

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4913
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1235
    })
})

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch

image_processor = AutoImageProcessor.from_pretrained("VaggP/convnext-tiny-finetuned-for-inpaintings-in-art-detection")
model = AutoModelForImageClassification.from_pretrained("VaggP/convnext-tiny-finetuned-for-inpaintings-in-art-detection")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

def transform(batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = image_processor([x for x in batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = batch['label']
    return inputs

dataset_inpaintings = dataset_inpaintings.with_transform(transform)

In [5]:
from torch.utils.data import DataLoader

import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

dataloader_test_inpaintings = DataLoader(dataset_inpaintings["test"], collate_fn=collate_fn, batch_size=8, shuffle=True)

In [6]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

total = 0
correct = 0
all_predicted = []
all_labels = []

for idx, batch in enumerate(tqdm(dataloader_test_inpaintings)):
    batch = {k:v.to(device) for k,v in batch.items()}

    with torch.no_grad():
        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])

    loss, logits = outputs.loss, outputs.logits

    predicted = logits.argmax(-1)
    correct += (predicted == batch["labels"]).sum().item()
    total += batch["labels"].shape[0]

    all_predicted.extend(predicted.cpu().numpy())
    all_labels.extend(batch["labels"].cpu().numpy())


accuracy = correct/total

precision = precision_score(all_labels, all_predicted)
recall = recall_score(all_labels, all_predicted)
f1 = f1_score(all_labels, all_predicted)

labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
conf_matrix = confusion_matrix(all_labels, all_predicted)

plt.show()
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")




  0%|          | 0/376 [00:00<?, ?it/s]

Accuracy: 0.8410
Precision: 0.7616
Recall: 0.9927
F1 Score: 0.8619
Confusion Matrix:
[[1036  467]
 [  11 1492]]


In [ ]:
image_processor = AutoImageProcessor.from_pretrained("VaggP/convnext-tiny-finetuned-for-style-transfer-in-art-detection")
model = AutoModelForImageClassification.from_pretrained("VaggP/convnext-tiny-finetuned-for-style-transfer-in-art-detection")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

dataset_style_transfer = dataset_style_transfer.with_transform(transform)

dataloader_test_style_transfer = DataLoader(dataset_style_transfer["test"], collate_fn=collate_fn, batch_size=8, shuffle=True)

In [8]:
total = 0
correct = 0
all_predicted = []
all_labels = []

for idx, batch in enumerate(tqdm(dataloader_test_style_transfer)):
    batch = {k:v.to(device) for k,v in batch.items()}

    with torch.no_grad():
        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])

    loss, logits = outputs.loss, outputs.logits

    predicted = logits.argmax(-1)
    correct += (predicted == batch["labels"]).sum().item()
    total += batch["labels"].shape[0]

    all_predicted.extend(predicted.cpu().numpy())
    all_labels.extend(batch["labels"].cpu().numpy())


accuracy = correct/total

precision = precision_score(all_labels, all_predicted)
recall = recall_score(all_labels, all_predicted)
f1 = f1_score(all_labels, all_predicted)

labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
conf_matrix = confusion_matrix(all_labels, all_predicted)

plt.show()
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

  0%|          | 0/155 [00:00<?, ?it/s]

Accuracy: 0.9895
Precision: 0.9882
Recall: 0.9899
F1 Score: 0.9890
Confusion Matrix:
[[635   7]
 [  6 587]]
